In [1]:
# ***************************************************************************************************
# Here you will find a simple version of a Variational Hybrid Quantum-Classical Algorithm based on a
# Parameterized Quantum Circuit (PQC).
# ***************************************************************************************************

# ***************************************************************************************************
# Code from:
# Zickert, F. (2021). Chapter 3: The Qubit and Quantum States
# In F. Zickert, Hands-On Quantum Machine Learning With Python: Volume 1: Get Started, PyQML, 2021. 
# Available on https://www.pyqml.com/
# ***************************************************************************************************

# ***************************************************************************************************
# Modifications have been made by Gustavo Patino.
# Engineering School
# University of Antioquia.
# Medellin, Colombia
# September, 2022.
# ***************************************************************************************************

In [2]:
# Listing 3.1 Verify Qiskit version
import qiskit
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram

import numpy as np
import pandas as pd
#import seaborn as sns # For pairplots and heatmaps
import matplotlib.pyplot as plt
from math import sqrt

In [3]:
from sklearn.metrics import recall_score, precision_score, confusion_matrix 
from qiskit_textbook.tools import vector2latex

In [4]:
# Descomente la siguiente linea si desea ver las imágenes más grandes.
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

In [5]:
# Importing the Qiskit lab. Drive:
import sys
sys.path.insert(0, '..')

In [6]:
# Listing 3.16: Load the data
with open('data/train.npy', 'rb') as f:
    train_input = np.load(f)
    train_labels = np.load(f)

with open('data/test.npy', 'rb') as f:
    test_input = np.load(f)
    test_labels = np.load(f)

In [7]:
# REDEFINE OR IMPORT THE FUNCTIONS OF CHAPTER 2
def run(f_classify, x):
    return list(map(f_classify, x))

def specificity(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

def classifier_report(name, run, classify, input, labels):
    cr_predictions = run(classify, input)
    cr_cm = confusion_matrix(labels, cr_predictions)

    cr_precision = precision_score(labels, cr_predictions)
    cr_recall = recall_score(labels, cr_predictions)
    cr_specificity = specificity(cr_cm)
    cr_npv = npv(cr_cm)
    cr_level = 0.25*(cr_precision + cr_recall + cr_specificity + cr_npv)

    print('\nThe precision score of the {} classifier is {:.2f}'
        .format(name, cr_precision))
    print('The recall score of the {} classifier is {:.2f}'
        .format(name, cr_recall))
    print('The specificity score of the {} classifier is {:.2f}'
        .format(name, cr_specificity))
    print('The npv score of the {} classifier is {:.2f}'
        .format(name, cr_npv))
    print('The information level is: {:.2f}'
        .format(cr_level))
#CAPTION A reusable function to unmask the hypocrite classifier

In [8]:
## The Random PQC

In [9]:
# Listing 3.20: Pre‐processing template
def pre_process_random(passenger):
    """
    passenger -- the normalized (array of numeric data) passenger data
    returns a valid quantum state
    """
    quantum_state = [1/sqrt(2), 1/sqrt(2)]
    
#   vector2latex(quantum_state, pretext="|\\psi\\rangle =")    
    
    return quantum_state

In [10]:
# Listing 3.21: The parameterized quantum circuit (PQC)
def pqc(backend, quantum_state):
    """
    backend -- a qiskit backend to run the quantum circuit at
    quantum_state -- a valid quantum state vector  
    returns the counts of the measurement
    """

    # Create a quantum circuit with one qubit
    qc = QuantumCircuit(1)

    # Define state |Psi> and initialize the circuit
    qc.initialize(quantum_state, 0)
    
    # Measure the qubit
    qc.measure_all()

    # run the quantum circuit
    result=execute(qc,backend).result()

    # get the counts, these are either {'0': 1} or {'1': 1}
    counts=result.get_counts(qc)
    
    print('Salida counts =', counts)

    return counts

In [11]:
# Listing 3.22: Post‐processing
def post_process(counts):
    """
    counts -- the result of the quantum circuit execution
    returns the prediction
    """
    
    Prediction = list(map(lambda item: item[0], counts.items()))[0]
    
    print('Given the former angle theta and quantum state, the new prediction is:')
    print('Prediction =', Prediction)
    print('int(Prediction) =', int(Prediction), '\n')
        
    return int(Prediction)

In [12]:
# Listing 3.23: The scores of the random quantum classifier
# Tell Qiskit how to simulate our circuit
backend = Aer.get_backend('statevector_simulator') 

In [13]:
classifier_report(
    "Random PQC",
    run,
    lambda passenger: post_process(pqc(backend, pre_process_random(passenger))),
    train_input,
    train_labels)

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Salida counts = {'1': 1}
Given the former angle theta and quantum sta

In [14]:
################################################
## Variational Hybrid Quantum-Classical Algorithm with Passenger Information
################################################

In [15]:
# Listing 3.24: weigh a passenger's feature
def weigh_feature(feature, weight):
    """
    feature -- the single value of a passenger's feature
    weight -- the overall weight of this feature
    returns the weighted feature 
    """
    return feature*weight

In [16]:
# Listing 3.25: Calculate the overall probability
from functools import reduce

def get_overall_probability(features, weights):
    """
    features -- list of the features of a passenger
    weights -- list of all features' weights
    """
    return reduce(lambda result, data: result + weigh_feature(*data), zip(features, weights),0) #result = result + weigh_feature(features, weights), starting at 0.

In [17]:
# separate the training data into a list of the columns
columns = [list(map(lambda passenger: passenger[i], train_input)) for i in range(0,7)]
#columns

In [18]:
# Listing 3.26: Calculate the correlation coefficients
from scipy.stats import spearmanr

# calculate the correlation coefficient for each column
correlations = list(map(lambda col: spearmanr(col, train_labels)[0], columns))
#correlations

In [19]:
dcorr = pd.DataFrame(correlations,index =['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'], columns = ['Ranking Correlation = Feature Weight'])
dcorr

,Ranking Correlation = Feature Weight
Pclass,-0.344766
Sex,-0.518456
Age,-0.025080
SibSp,0.095559
Parch,0.127698
Fare,0.310622
Embarked,-0.184166


In [20]:
# Listing 3.27: The weighting pre‐processing
from math import pi, sin, cos

def get_state (theta):
    """returns a valid state vector from angle theta"""
    return [cos(theta/2), sin(theta/2)]

In [21]:
def pre_process_weighted(passenger):
    """
    passenger -- the normalized (array of numeric data) passenger data
    returns a valid quantum state
    """

    # caluclate the overall probability
    mu = get_overall_probability(passenger, correlations)
    print('Overall Probability =', mu)
        
    # theta between 0 (|0>) and pi (|1>)
    quantum_state = get_state((1-mu)*pi)
    
    vector2latex(quantum_state, pretext="|\\psi\\rangle =")  

    return quantum_state

In [22]:
backend = Aer.get_backend('statevector_simulator') 

In [23]:
# Listing 3.28: Run the PQC with the weighted pre‐processing
classifier_report("Variational Hybrid Quantum Circuit with Data Train", 
    run,
    lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))),
    train_input,
    train_labels)

Overall Probability = -0.9598151743109983


<IPython.core.display.Math object>

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Overall Probability = 0.0915700222443043


<IPython.core.display.Math object>

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Overall Probability = -1.0387423503683644


...

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Overall Probability = -0.8661121426104289


<IPython.core.display.Math object>

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Overall Probability = -0.8735727646485079


<IPython.core.display.Math object>

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 


The precision score of the Variational Hybrid Quantum Circuit with Data Train classifier is 0.70
The recall score of the Variational Hybrid Quantum Circuit with Data Train classifier is 0.62
The specificity score of the Variational Hybrid Quantum Circuit with Data Train classifier is 0.84
The npv score of the Variational Hybrid Quantum Circuit with Data Train classifier is 0.79
The information level is: 0.74


In [24]:
# Listing 3.29: Test the PQC‐based classifier on data it has not seen before
classifier_report("Variational HQC with Data Test", 
    run,
    lambda passenger: post_process(pqc(backend, pre_process_weighted(passenger))),
    test_input,
    test_labels)

Overall Probability = -0.36076580294684757


<IPython.core.display.Math object>

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Overall Probability = 0.042511471829814275


<IPython.core.display.Math object>

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Overall Probability = -0.8598923077481491


...

Salida counts = {'0': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 0
int(Prediction) = 0 

Overall Probability = -0.41388707064416436


<IPython.core.display.Math object>

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 

Overall Probability = -0.1363380007417963


<IPython.core.display.Math object>

Salida counts = {'1': 1}
Given the former angle theta and quantum state, the new prediction is:
Prediction = 1
int(Prediction) = 1 


The precision score of the Variational HQC with Data Test classifier is 0.81
The recall score of the Variational HQC with Data Test classifier is 0.55
The specificity score of the Variational HQC with Data Test classifier is 0.90
The npv score of the Variational HQC with Data Test classifier is 0.72
The information level is: 0.74


In [25]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright